In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeledTrainData.tsv'
UNLABELED_TRAIN_DF = BASE_DIR + 'unlabeledTrainData.tsv'
TEST_DF = BASE_DIR + 'testData.tsv'
print(os.listdir(BASE_DIR))

['word2vec_model_300dim_40minwords_10context', 'sampleSubmission.csv', 'word2vec_model_300dim_40minwords_10context_stemmed', 'labeledTrainData.tsv', 'test_submission.csv', 'testData.tsv', 'unlabeledTrainData.tsv']


In [2]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
unlabeled_train = pd.read_csv(UNLABELED_TRAIN_DF, header = 0, delimiter = '\t', quoting=3)
test = pd.read_csv(TEST_DF, header = 0, delimiter = '\t', quoting=3)
print "Read %d labeled train reviews, %d unlabeled train reviews, " \
          "and %d test reviews" % (labeled_train["review"].size, unlabeled_train["review"].size, test["review"].size)

Read 25000 labeled train reviews, 50000 unlabeled train reviews, and 25000 test reviews


Data leakage

Check if test["sentiment"] is correct 

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = test["sentiment"]

Credits: Kaggle tutorial

In [6]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def review_to_clean_review(review, remove_numbers=True, stem_words=False):
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z0-9]", " ", review_text)
    
    if remove_numbers:
        review_text = re.sub("[0-9]", " ", review_text)
    else:
        review_text = review_text.replace('0', ' zero ')
        review_text = review_text.replace('1', ' one ')
        review_text = review_text.replace('2', ' two ')
        review_text = review_text.replace('3', ' three ')
        review_text = review_text.replace('4', ' four ')
        review_text = review_text.replace('5', ' five ')
        review_text = review_text.replace('6', ' six ')
        review_text = review_text.replace('7', ' seven ')
        review_text = review_text.replace('8', ' eight ')
        review_text = review_text.replace('9', ' nine ')
    
    review_text = review_text.lower()
    
    if stem_words:
        words = review_text.split()
        stemmer = SnowballStemmer('english')
        words = [stemmer.stem(word) for word in words]
        review_text = " ".join(words)
        
    return(review_text)

def review_to_wordlist(review, remove_stopwords=False, remove_numbers=True, stem_words=False):

    words = review_to_clean_review(review, remove_numbers, stem_words).split()

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False, remove_numbers=True, stem_words=False):
    
    raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords, remove_numbers, stem_words))
    return sentences

In [7]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Initialize an empty list of sentences
sentences = []
# Parsing sentences from training set
counter = 0.
for review in labeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer, remove_numbers=False, stem_words=True)
    if counter % 5000. == 0.:
        print "Labled train review %d of %d" % (counter, len(labeled_train["review"]))
    counter = counter + 1.

counter = 0.
# Parsing sentences from unlabeled set
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer, remove_numbers=False, stem_words=True)
    if counter % 5000. == 0.:
        print "Unlabled train review %d of %d" % (counter, len(unlabeled_train["review"]))
    counter = counter + 1.

Labled train review 0 of 25000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:272: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Labled train review 5000 of 25000
Labled train review 10000 of 25000
Labled train review 15000 of 25000
Labled train review 20000 of 25000
Unlabled train review 0 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Unlabled train review 5000 of 50000
Unlabled train review 10000 of 50000
Unlabled train review 15000 of 50000
Unlabled train review 20000 of 50000


...

/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:272: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Unlabled train review 35000 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://www.youtube.com/watch?v=a0KSqelmgN8"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Unlabled train review 40000 of 50000
Unlabled train review 45000 of 50000


/home/oscar/.local/lib/python2.7/site-packages/bs4/__init__.py:335: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [13]:
# print(np.random.choice(sentences, 10))

In [15]:
output_texts = [" ".join(sentence) for sentence in sentences]
print(np.random.choice(output_texts, 10))

[u'this is strang to say the least as everyon can clear see that uma thurman doe not belong under the categori of hot rainn wilson s perform is also far from hot'
 u'especi sinc the end result is their hair be straighten'
 u'but then whi not just get up and walk across the countri to watch tv in anoth state'
 u'the film also has a veri good sens of humour and it made me laugh'
 u'this vignett deal with the fragil hold of saniti on the conscious mind'
 u'the perform were brilliant it is question whi no nomin were offer'
 u'and at least those show were meant for children this show howev is by far wors'
 u'the cast is surpris good the plot move well and the contrast involv in the film are strike there are two main subplot which revolv around the one nine four one pearl harbor attack first is the relationship between mp colonel jason forrest his wife and vari other under forrest s command notabl his xo captain calvin lanford a particular effect perform by robert wagner'
 u'i first saw this

In [ ]:
output_texts = ["".join(sentence) for sentence in sentences]
output = pd.DataFrame(data={"id": test["id"], "sentiment": sentences})
output.to_csv(os.path.join(os.path.dirname(__file__), '../', 'output', "Word2Vec_AverageVectors.csv"), index=False, quoting=3)
print "Wrote clean.csv"

Training the Word2Vec model

In [11]:
# Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)

from gensim.models import Word2Vec
EMBEDDING_DIM = 300  # Word vector dimensionality
MIN_WORD_COUNT = 40  # Minimum word count. Kaggle set to 40, to avoid attaching too much importance to individual movie titles.
NUM_THREADS = 4  # Number of threads to run in parallel
CONTEXT = 10  # Context window size
DOWNSAMPLING = 1e-3  # Downsample setting for frequent words
WORD2VEC_MODEL_FILE = BASE_DIR + \
    "word2vec_model_" + \
    str(EMBEDDING_DIM) + "dim_" + \
    str(MIN_WORD_COUNT) + "minwords_" + \
    str(CONTEXT) + "context_" +\
    "stemmed"

print "Training the Word2Vec model..."
model = Word2Vec(sentences, workers=NUM_THREADS, \
                 size=EMBEDDING_DIM, min_count=MIN_WORD_COUNT, \
                 window=CONTEXT, sample=DOWNSAMPLING, seed=1)
model.save(WORD2VEC_MODEL_FILE)


2019-01-06 06:29:16,103 : INFO : collecting all words and their counts
2019-01-06 06:29:16,105 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-01-06 06:29:16,178 : INFO : PROGRESS: at sentence #10000, processed 229166 words, keeping 12465 word types
2019-01-06 06:29:16,236 : INFO : PROGRESS: at sentence #20000, processed 458297 words, keeping 17070 word types


Training the Word2Vec model...


2019-01-06 06:29:16,293 : INFO : PROGRESS: at sentence #30000, processed 680652 words, keeping 20370 word types
2019-01-06 06:29:16,354 : INFO : PROGRESS: at sentence #40000, processed 910089 words, keeping 23125 word types
2019-01-06 06:29:16,412 : INFO : PROGRESS: at sentence #50000, processed 1132423 words, keeping 25365 word types
2019-01-06 06:29:16,471 : INFO : PROGRESS: at sentence #60000, processed 1356968 words, keeping 27283 word types
2019-01-06 06:29:16,529 : INFO : PROGRESS: at sentence #70000, processed 1583238 words, keeping 29024 word types
2019-01-06 06:29:16,588 : INFO : PROGRESS: at sentence #80000, processed 1805500 words, keeping 30603 word types
2019-01-06 06:29:16,648 : INFO : PROGRESS: at sentence #90000, processed 2032837 words, keeping 32223 word types
2019-01-06 06:29:16,708 : INFO : PROGRESS: at sentence #100000, processed 2257707 words, keeping 33579 word types
2019-01-06 06:29:16,768 : INFO : PROGRESS: at sentence #110000, processed 2480677 words, keeping 

2019-01-06 06:29:20,719 : INFO : PROGRESS: at sentence #750000, processed 17001852 words, keeping 84599 word types
2019-01-06 06:29:20,778 : INFO : PROGRESS: at sentence #760000, processed 17224202 words, keeping 85068 word types
2019-01-06 06:29:20,842 : INFO : PROGRESS: at sentence #770000, processed 17454566 words, keeping 85644 word types
2019-01-06 06:29:20,904 : INFO : PROGRESS: at sentence #780000, processed 17687962 words, keeping 86160 word types
2019-01-06 06:29:20,966 : INFO : PROGRESS: at sentence #790000, processed 17918070 words, keeping 86665 word types
2019-01-06 06:29:21,002 : INFO : collected 86996 word types from a corpus of 18042771 raw words and 795538 sentences
2019-01-06 06:29:21,004 : INFO : Loading a fresh vocabulary
2019-01-06 06:29:21,069 : INFO : effective_min_count=40 retains 11986 unique words (13% of original 86996, drops 75010)
2019-01-06 06:29:21,070 : INFO : effective_min_count=40 leaves 17678536 word corpus (97% of original 18042771, drops 364235)
201

2019-01-06 06:30:13,116 : INFO : EPOCH 3 - PROGRESS: at 66.82% examples, 668689 words/s, in_qsize 8, out_qsize 0
2019-01-06 06:30:14,122 : INFO : EPOCH 3 - PROGRESS: at 71.99% examples, 669264 words/s, in_qsize 8, out_qsize 0
2019-01-06 06:30:15,129 : INFO : EPOCH 3 - PROGRESS: at 77.20% examples, 669754 words/s, in_qsize 8, out_qsize 0
2019-01-06 06:30:16,132 : INFO : EPOCH 3 - PROGRESS: at 82.35% examples, 669829 words/s, in_qsize 8, out_qsize 0
2019-01-06 06:30:17,142 : INFO : EPOCH 3 - PROGRESS: at 87.54% examples, 670112 words/s, in_qsize 8, out_qsize 0
2019-01-06 06:30:18,145 : INFO : EPOCH 3 - PROGRESS: at 92.74% examples, 670627 words/s, in_qsize 8, out_qsize 0
2019-01-06 06:30:19,151 : INFO : EPOCH 3 - PROGRESS: at 97.87% examples, 670597 words/s, in_qsize 8, out_qsize 0
2019-01-06 06:30:19,533 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-01-06 06:30:19,549 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-01-06 06:30:19,551 : I